In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import subprocess
import os
import psutil
import signal
import random

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
running_processes = []

[nltk_data] Downloading package punkt to /home/ambhi/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/ambhi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Training Data

In [4]:
training_data = [
    ("Can I join the meeting room?", "joining"),
    ("Is there any available room?", "joining"),
    ("Can I be part of this room?", "joining"),
    ("I want to leave the conference room.", "leaving"),
    ("Leaving the room now.", "leaving"),
    ("I'm done with this meeting room.", "leaving"),
    ("Create a new chat room.", "creating"),
    ("Can you help me create a room?", "creating"),
    ("Let's create a private room for our team.", "creating"),
]

In [5]:
additional_training_data = [
    ("How do I join the video call?", "joining"),
    ("Is there a way to enter the meeting room?", "joining"),
    ("Can you add me to the group?", "joining"),
    ("Exiting the virtual conference.", "leaving"),
    ("I need to leave the call now.", "leaving"),
    ("Leaving the chat room.", "leaving"),
    ("Create a new public chat room.", "creating"),
    ("Let's create a channel for discussions.", "creating"),
    ("I want to create a separate room for this project.", "creating"),
    ("How do I mute myself in the meeting?", "joining"),
    ("I want to mute my audio during the conference.", "joining"),
    ("How do I turn off my video?", "joining"),
    ("Can we end the call now?", "leaving"),
    ("I'm ending the meeting.", "leaving"),
    ("Let's finish this conference.", "leaving"),
    ("Create a new voice call room.", "creating"),
    ("Can you make a room for a one-on-one call?", "creating"),
    ("I need to set up a meeting room for the team.", "creating"),
    ("How do I share my screen?", "joining"),
    ("Can you show me how to share the screen in the meeting?", "joining"),
    ("I want to share my screen with the group.", "joining"),
]
additional_training_data1 = [
    ("How do I delete a chatroom?", "deleting"),
    ("Is there a way to remove a chatroom?", "deleting"),
    ("Can you help me delete a group?", "deleting"),
    ("Closing the chatroom.", "deleting"),
    ("I need to remove this chatroom.", "deleting"),
    ("Deleting the group chat.", "deleting"),
    ("How can I archive a chatroom?", "deleting"),
    ("Can you show me the steps to delete a chatroom?", "deleting"),
    ("I want to close this room permanently.", "deleting"),
]

training_data = training_data + additional_training_data + additional_training_data1
random.shuffle(training_data)


#### Preprocessing Data and Training

In [6]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return stemmed_tokens

def extract_features(text):
    return {word: True for word in preprocess_text(text)}

training_set = [(extract_features(text), intent) for text, intent in training_data]
classifier = nltk.NaiveBayesClassifier.train(training_set)


In [7]:
def predict_intent(text):
    features = extract_features(text)
    return classifier.classify(features)

#### Subprocesses

In [8]:
def run_ffmpeg_command(room_name, video_sock, audio_sock):
    command = ['./ffmpeg-connect.sh', video_sock, audio_sock]
    try:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        running_processes.append(process.pid)
        print(f"{room_name} - Video and audio connected.")
    except subprocess.CalledProcessError as e:
        print(f"Error connecting video and audio for {room_name}: {e.stderr.decode('utf-8')}")

def join_room_script(room_name, video_sock, audio_sock):
    command = ['./join-room-script.sh', room_name, video_sock, audio_sock]
    try:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        running_processes.append(process.pid)
        print(f"{room_name} - Joined the room.")
    except subprocess.CalledProcessError as e:
        print(f"Error joining the room {room_name}: {e.stderr.decode('utf-8')}")

def create_room_script(room_name):
    command = ['./create-room-script.sh',room_name]
    try:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        running_processes.append(process.pid)
        print(f"{room_name} - Room Creating")
    except subprocess.CalledProcessError as e:
        print(f"{room_name} - Error joining the room")

def delete_room_script(room_name):
    command = ['./delete-room-script.sh', room_name]
    try:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        running_processes.append(process.pid)
        print(f"{room_name} - Deleted")
    except subprocess.CalledProcessError as e:
        print(f"Error deleting the room {room_name}: {e.stderr.decode('utf-8')}")


In [9]:
def kill_process_by_name(process_name):
    for process in psutil.process_iter(['pid', 'name']):
        if process.info['name'] == process_name:
            try:
                os.kill(process.pid, signal.SIGKILL)
                process.terminate()
            except psutil.NoSuchProcess:
                pass  
            except psutil.AccessDenied:
                print(f"Access denied to terminate process '{process_name}'.")
            else:
                print(f"Process '{process_name}' terminated.")

In [13]:
def handle_joining():
    run_ffmpeg_command("TheView","/tmp/myvideo.sock","/tmp/myaudio.sock")
    join_room_script("TheView","/tmp/myvideo.sock","/tmp/myaudio.sock")
    print("Joined!!")

def handle_creating():
    create_room_script("TheView")
    print("Created")

def handle_leaving():
    kill_process_by_name("livekit-cli")
    kill_process_by_name("ffmpeg-connect.sh")
    kill_process_by_name("join-room-script.sh")
    running_processes.clear()
    print("Left")

def handle_deleting():
    delete_room_script("TheView")
    handle_leaving()
    kill_process_by_name("delete-room-script.sh")
    print("Deleted")


def invoke_command(predicted_intent):
    
    if predicted_intent == "joining":
        handle_joining()
    elif predicted_intent == "creating":
        handle_creating()
    elif predicted_intent == "leaving":
        handle_leaving()
    elif predicted_intent == "deleting":
        handle_deleting()
    else:
        print("I do not understand!")

#### Input

In [14]:
while True:
    user_input = input("Enter prompt: ")
    if user_input == "break":
        break
    predicted_intent = predict_intent(user_input)
    print("Predicted Intent:", predicted_intent)
    invoke_command(predicted_intent)

Predicted Intent: joining
TheView - Video and audio connected.
TheView - Joined the room.
Joined!!
Predicted Intent: leaving
Process 'livekit-cli' terminated.
Process 'livekit-cli' terminated.
Left
Predicted Intent: leaving
Left
Predicted Intent: deleting
TheView - Deleted
Left
Deleted
Predicted Intent: creating
TheView - Room Creating
Created
Predicted Intent: joining
TheView - Video and audio connected.
TheView - Joined the room.
Joined!!
Predicted Intent: deleting
TheView - Deleted
Process 'livekit-cli' terminated.
Left
Deleted
